## Import Functions and Spark Config

In [4]:
%run ./read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-cab94300-49f5-427b-9430-76062ab33a35;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.2-spark3.2-s_2.12!graphframes.jar (42ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (27ms)
:: resolution report :: resolve 744ms :: artifacts dl 74ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	--------

23/03/03 02:57:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/03 02:57:14 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [5]:
print('PySpark Version :'+spark.version)

PySpark Version :3.3.0


In [6]:
# Create a Vertex DataFrame with unique ID column "id"
v = spark.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
], ["id", "name", "age"])
# Create an Edge DataFrame with "src" and "dst" columns
e = spark.createDataFrame([
  ("a", "b", "friend"),
  ("b", "c", "follow"),
  ("c", "b", "follow"),
], ["src", "dst", "relationship"])
# Create a GraphFrame
from graphframes import *
g = GraphFrame(v, e)

# Query: Get in-degree of each vertex.
g.inDegrees.show()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+---+--------+
| id|inDegree|
+---+--------+
|  b|       2|
|  c|       1|
+---+--------+



In [7]:
# Query: Count the number of "follow" connections in the graph.
g.edges.filter("relationship = 'follow'").count()

2

In [8]:
# Run PageRank algorithm, and show results.
results = g.pageRank(resetProbability=0.01, maxIter=20)
results.vertices.select("id", "pagerank").show()

+---+------------------+
| id|          pagerank|
+---+------------------+
|  c|1.8994109890559092|
|  b|1.0905890109440908|
|  a|              0.01|
+---+------------------+



## Set Environment

In [2]:
# env - dev / prod
# size - small / medium / large / all
# readCheckpoint automatically caches. use df.unpersist() to release from memory.
env='prod'
size='small'

## Data Processing

In [3]:
df_ecar = readCheckpoint('ecar', env, size).cache()

 8:45AM UTC on Feb 28, 2023 --- read and cache time: 13.244666814804077 seconds ---


In [4]:
df_ecar.limit(25).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,action,actorID,hostname,id,object,objectID,pid,ppid,principal,tid,...,tgt_tid,type,user,user_name,user_stack_base,user_stack_limit,value,event_day,event_hour,event_minute
0,INFO,c183f6f4-e83e-4d05-98df-6cafce2ec010,SysClient0102.systemia.com,b7710554-e7e6-4957-9d97-6507a557e11d,FLOW,eeee9e56-df6e-476c-bfc7-03d507dbd8e7,3868,4172,SYSTEMIACOM\jnyholm,-1,...,NaN,None,None,None,None,None,None,23,13,11
1,INFO,c183f6f4-e83e-4d05-98df-6cafce2ec010,SysClient0102.systemia.com,f31e40f3-620c-4ee0-bff8-6e3ecf150558,FLOW,c7bfc171-627d-4103-b883-d5c3d4fd2f4d,3868,4172,SYSTEMIACOM\jnyholm,-1,...,NaN,None,None,None,None,None,None,23,13,11
2,INFO,c183f6f4-e83e-4d05-98df-6cafce2ec010,SysClient0102.systemia.com,41514109-13b4-4d2a-a924-f1e15081178a,FLOW,da018e32-281d-4a74-83cc-c113fc70900b,3868,4172,SYSTEMIACOM\jnyholm,-1,...,NaN,None,None,None,None,None,None,23,13,11
3,INFO,c183f6f4-e83e-4d05-98df-6cafce2ec010,SysClient0102.systemia.com,c23750d0-292d-48cb-97da-00145df91ee4,FLOW,b48bef19-32d5-4727-9a29-4e0186dcffe0,3868,4172,SYSTEMIACOM\jnyholm,-1,...,NaN,None,None,None,None,None,None,23,13,11
4,INFO,c183f6f4-e83e-4d05-98df-6cafce2ec010,SysClient0102.systemia.com,fc8bc743-620b-4236-9ad2-d95082056bef,FLOW,cd7be00d-3691-484c-aedb-954582d6353c,3868,4172,SYSTEMIACOM\jnyholm,-1,...,NaN,None,None,None,None,None,None,23,13,11
5,INFO,c183f6f4-e83e-4d05-98df-6cafce2ec010,SysClient0102.systemia.com,32e31bb7-1f32-4116-a5c9-423ff5aaeae4,FLOW,6e1dc787-f33c-440f-b019-767097b01239,3868,4172,SYSTEMIACOM\jnyholm,-1,...,NaN,None,None,None,None,None,None,23,13,11
6,INFO,c183f6f4-e83e-4d05-98df-6cafce2ec010,SysClient0102.systemia.com,d274d7d7-80b0-4b3d-b6c8-fc2a016aff46,FLOW,42172162-c675-48af-a5f7-40b4ea15c8e3,3868,4172,SYSTEMIACOM\jnyholm,-1,...,NaN,None,None,None,None,None,None,23,13,11
7,INFO,c183f6f4-e83e-4d05-98df-6cafce2ec010,SysClient0102.systemia.com,db41913d-c6bd-480d-88a0-af84b44a85f8,FLOW,1e64ec11-584d-459b-9e42-01776352e103,3868,4172,SYSTEMIACOM\jnyholm,-1,...,NaN,None,None,None,None,None,None,23,13,11
8,INFO,c183f6f4-e83e-4d05-98df-6cafce2ec010,SysClient0102.systemia.com,8b347d12-ad5a-43aa-9e98-8f292719b09c,FLOW,79a5cd8b-54ec-4f7f-bde7-2a4c1abd3f5c,3868,4172,SYSTEMIACOM\jnyholm,-1,...,NaN,None,None,None,None,None,None,23,13,11
9,INFO,c183f6f4-e83e-4d05-98df-6cafce2ec010,SysClient0102.systemia.com,4155afd1-e186-4bfe-98e4-4cd12731e2eb,FLOW,2cc1204c-924e-472d-bc44-1cdaddcbc8e4,3868,4172,SYSTEMIACOM\jnyholm,-1,...,NaN,None,None,None,None,None,None,23,13,11


In [5]:
df_labels = readCheckpoint('labels', env)

 8:45AM UTC on Feb 28, 2023 --- read and cache time: 0.2422952651977539 seconds ---


In [6]:
df_labels.limit(5).toPandas()

,hostname,id,objectID,actorID,timestamp,object,action
0,SysClient0955.systemia.com,3e350ce3-c4b3-4640-92d9-feaea0274cab,83f11800-88a3-4fa9-8c51-a469d66fca16,473d7954-6271-43b8-9984-702378220af2,2019-09-23T14:58:28.051-04:00,SHELL,COMMAND
1,SysClient0955.systemia.com,7a396cde-6485-40b4-baa2-8d7848fa02a3,83f11800-88a3-4fa9-8c51-a469d66fca16,473d7954-6271-43b8-9984-702378220af2,2019-09-23T14:58:33.139-04:00,SHELL,COMMAND
2,SysClient0955.systemia.com,2f33f5e3-1714-4069-850f-863680edbdef,83f11800-88a3-4fa9-8c51-a469d66fca16,473d7954-6271-43b8-9984-702378220af2,2019-09-23T14:58:33.141-04:00,SHELL,COMMAND
3,SysClient0955.systemia.com,725ad9a5-c876-4dc2-9c32-12dc6bb4da66,83f11800-88a3-4fa9-8c51-a469d66fca16,473d7954-6271-43b8-9984-702378220af2,2019-09-23T14:58:33.159-04:00,SHELL,COMMAND
4,SysClient0955.systemia.com,a8a78df2-a996-4c16-90f7-6455091eb91d,83f11800-88a3-4fa9-8c51-a469d66fca16,473d7954-6271-43b8-9984-702378220af2,2019-09-23T14:58:33.158-04:00,SHELL,COMMAND


In [7]:
df_labels.select(countDistinct("objectID")).show()

+------------------------+
|count(DISTINCT objectID)|
+------------------------+
|                   69876|
+------------------------+



In [8]:
df_labels.select(countDistinct("actorID")).show()

+-----------------------+
|count(DISTINCT actorID)|
+-----------------------+
|                    150|
+-----------------------+



In [9]:
df_labels.select("object").distinct().show()

+--------+
|  object|
+--------+
|   SHELL|
|REGISTRY|
|    FLOW|
|  THREAD|
|    FILE|
|  MODULE|
| PROCESS|
+--------+



In [10]:
df_labels.groupby("object","id").count().groupBy("object").count().orderBy(col("count").desc()).show()

+--------+------+
|  object| count|
+--------+------+
|   SHELL|165473|
|    FLOW| 80436|
|    FILE| 22485|
|  THREAD| 11283|
| PROCESS|  7908|
|  MODULE|  4570|
|REGISTRY|   109|
+--------+------+



In [11]:
df_labels.groupby("action","id").count().groupBy("action").count().orderBy(col("count").desc()).show()

+-------------+------+
|       action| count|
+-------------+------+
|      COMMAND|165473|
|      MESSAGE| 55781|
|        START| 24652|
|         READ| 10567|
|       CREATE|  8967|
|         OPEN|  7218|
|    TERMINATE|  6555|
|        WRITE|  5079|
|         LOAD|  4570|
|       RENAME|  1846|
|REMOTE_CREATE|   614|
|       MODIFY|   515|
|       DELETE|   318|
|         EDIT|    61|
|          ADD|    31|
|       REMOVE|    17|
+-------------+------+



In [12]:
df_labels.groupby("hostname","actorID","id").count().groupBy("hostname","actorID").count().orderBy(col("count").desc()).show()

+--------------------+--------------------+-----+
|            hostname|             actorID|count|
+--------------------+--------------------+-----+
|SysClient0201.sys...|a52f6519-6265-443...|25809|
|SysClient0010.sys...|3a3de01b-44d0-4a1...|19453|
|SysClient0069.sys...|a80f73dc-5713-460...|19438|
|SysClient0203.sys...|c2730252-3ab3-40f...|19433|
|SysClient0358.sys...|f247d0d1-6ce4-4f8...|19398|
|SysClient0851.sys...|266392d7-012e-479...|19352|
|SysClient0618.sys...|9211f414-ef00-4aa...|19274|
|SysClient0811.sys...|60f28094-a2f7-49e...|17706|
|SysClient0351.sys...|903916d0-e580-45e...|16857|
|SysClient0501.sys...|054df7d9-801e-49d...|13673|
|SysClient0501.sys...|281edc04-1ee7-4f6...|10561|
|SysClient0660.sys...|3eea9071-9ceb-468...| 9189|
|SysClient0402.sys...|e490633e-4e82-4bb...| 8065|
|SysClient0355.sys...|d359ffa5-8bf5-4f1...| 4621|
|SysClient0559.sys...|53efe8c9-3c95-4df...| 4396|
|SysClient0255.sys...|b5a0a459-229f-4e4...| 4393|
|SysClient0874.sys...|ee99fcf7-84fa-49f...| 4392|


In [13]:
df_ecar.select("acuity_level").distinct().show()

+------------+
|acuity_level|
+------------+
|        null|
+------------+



In [ ]:
nodes = df_ecar.select(